## Imports

In [9]:
import sys
sys.path.append("..")

import re
from collections import OrderedDict
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import func, select, desc, extract
from sqlalchemy.sql.expression import and_

import config
from cyclops.orm import Database
from cyclops.processors.utils import get_icd_category

## Load config, setup ORM

In [2]:
cfg = config.read_config("../configs/default/*.yaml")
db = Database(cfg)

2022-02-16 10:45:14,190 INFO  cyclops.orm     - Database setup, ready to run queries!


## Extract slices

In [23]:
HOSPITAL = "SMH"
YEAR = 2020
LIMIT_LABS = 100000
LIMIT_MEDICATIONS = 100000
LIMIT_VITALS = 100000

## Query to get admin + diagnosis

In [16]:
query = select(db.public.ip_administrative.x,
               db.public.diagnosis.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.diagnosis.x, 
                    db.public.ip_administrative.genc_id == db.public.diagnosis.genc_id
                   )
              
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 11:00:12,087 INFO  cyclops.orm     - Query returned successfully, took 0.6026113033294678 s


15840 rows fetched!


## Query to get admin + labs

In [20]:
query = select(db.public.ip_administrative.x,
               db.public.lab.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.lab.x, 
                    db.public.ip_administrative.genc_id == db.public.lab.genc_id
                   ).limit(LIMIT_LABS) # Use limit here, since lab queries can have millions of rows!
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 11:00:46,619 INFO  cyclops.orm     - Query returned successfully, took 12.74314546585083 s


100000 rows fetched!


## Query to get admin + pharmacy

In [22]:
query = select(db.public.ip_administrative.x,
               db.public.pharmacy.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.pharmacy.x, 
                    db.public.ip_administrative.genc_id == db.public.pharmacy.genc_id
                   ).limit(LIMIT_MEDICATIONS) # Use limit here, since pharmacy queries can have millions of rows!
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 11:03:20,572 INFO  cyclops.orm     - Query returned successfully, took 13.430439472198486 s


76421 rows fetched!


## Query to get admin + vitals

In [24]:
query = select(db.public.ip_administrative.x,
               db.public.vitals.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.vitals.x, 
                    db.public.ip_administrative.genc_id == db.public.vitals.genc_id
                   ).limit(LIMIT_VITALS)
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 11:03:59,590 INFO  cyclops.orm     - Query returned successfully, took 4.854304790496826 s


100000 rows fetched!
